In [6]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
import cvzone
from datetime import datetime
import pytz

model=YOLO('yolov8s.pt')

In [7]:
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
    

In [8]:

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('shoppingmall.mp4')

# Verifique se o vídeo foi aberto corretamente
if not cap.isOpened():
    print("Erro ao abrir o vídeo.")
    exit()

# Obtenha as informações de shape e FPS do vídeo
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Obtenha o fuso horário do Brasil
timezone = pytz.timezone('America/Sao_Paulo')
# Obtenha a data e hora atual no fuso horário do Brasil
data_hora_atual = datetime.now(timezone)
data_hora_formatada = data_hora_atual.strftime("%d/%m/%Y - %H:%M:%S")

output_file = 'output.mp4'
execution_file = 'execution.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width,frame_height))
execution_out = cv2.VideoWriter(execution_file, fourcc, fps, (frame_width,frame_height))

In [9]:
my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

In [ ]:
count=0

tracker=Tracker()
#desenhando as áreas
area1=[(708,239),(690,253),(945,334),(959,317)] #vermelha
area2=[(681,257),(677,265),(927,353),(937,342)] #verde

people_enter={}
counter1=[]

people_exit={}
counter2=[]
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    #frame=cv2.resize(frame,(frame_width,frame_height))
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]         
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'person' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id: #se uma pessoa foi detectada
        x3,y3,x4,y4,id=bbox 
        results = cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
        #1 se a pessoa esta dentro da área. -1 se não estiver
        if results>=0: #Caso a pessoa esteja dentro da área, nós mostramos o ponto e add ela a lista. 
            cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,255),1)
            cv2.circle(frame,(x4,y4),4,(255,0,0),-1) #Fazendo a bolinha no canto direito
            cvzone.putTextRect(frame,f'{id}',(x3,y3),1,2)
            people_exit[id]=(x4,y4) #se a pessoa passou pela área 1, nos guardamos a posição. 
            #Para a área 2, conte apenas as pessoas que já passarem pela área 1
            
        if id in people_exit:
            results1 = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
            if results1 >=0:
                cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,255),1)
                cv2.circle(frame,(x4,y4),4,(255,0,0),-1) #Fazendo a bolinha no canto direito
                cvzone.putTextRect(frame,f'{id}',(x3,y3),1,2)
                if counter2.count(id)==0:  #testando para ver se a ID já está na lista. Assim ñ duplicamos
                    counter2.append(id) 

        #Contagem de Pessoas Entrando            
        results2 = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False)
        #1 se a pessoa esta dentro da área. -1 se não estiver
        if results2>=0: #area
            cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,255),1)
            cv2.circle(frame,(x4,y4),4,(255,0,0),-1) #Fazendo a bolinha no canto direito
            cvzone.putTextRect(frame,f'{id}',(x3,y3),1,2)
            people_enter[id]=(x4,y4) #se a pessoa passou pela área 1, nos guardamos a posição. 
            #Para a área 2, conte apenas as pessoas que já passarem pela área 1
            
        if id in people_enter:
            results3 = cv2.pointPolygonTest(np.array(area1,np.int32),((x4,y4)),False)
            if results3 >=0:
                cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,255),1)
                cv2.circle(frame,(x4,y4),4,(255,0,0),-1) #Fazendo a bolinha no canto direito
                cvzone.putTextRect(frame,f'{id}',(x3,y3),1,2)
                if counter1.count(id)==0:
                    counter1.append(id) 

        
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(0,0,255),1) #area vermelha
    cv2.polylines(frame,[np.array(area2,np.int32)],True,(0,255,0),1) #area verde   
    er = len(counter1) #contando o total da lista de entra e saida
    et = len(counter2) 
    cvzone.putTextRect(frame,f'{data_hora_formatada}',(50,20),2,2)
    cvzone.putTextRect(frame,f'Entrando: {er}',(50,50),2,2) #printando na tela
    cvzone.putTextRect(frame,f'Saindo: {et}',(50,80),2,2)
    

    out.write(frame)
    execution_out.write(frame)
        
    cv2.imshow("RGB", frame)
    if cv2.waitKey(0)&0xFF==27:
        break
cap.release()
out.release()
execution_out.release()
cv2.destroyAllWindows()


0: 320x640 (no detections), 481.2ms
Speed: 3.7ms preprocess, 481.2ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


[5, 270]
[8, 270]
[11, 270]
[17, 270]
[23, 270]
[24, 270]
[29, 270]
[43, 271]
[64, 271]
[86, 275]
[112, 277]
[138, 277]
[166, 273]
[190, 271]
[212, 271]
[234, 271]
[254, 273]
[274, 273]
[284, 273]
[290, 273]
[294, 273]
[298, 275]
[299, 275]
[300, 275]
[301, 276]
[307, 277]
[316, 276]



0: 320x640 (no detections), 340.5ms
Speed: 2.7ms preprocess, 340.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 381.9ms
Speed: 5.0ms preprocess, 381.9ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 341.5ms
Speed: 3.3ms preprocess, 341.5ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 354.9ms
Speed: 2.8ms preprocess, 354.9ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 (no detections), 351.5ms
Speed: 5.0ms preprocess, 351.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' inst

[317, 276]
[318, 276]
[318, 276]
[321, 275]
[327, 272]
[329, 272]
[330, 271]
[330, 270]
[331, 270]


0: 320x640 1 person, 1 suitcase, 354.7ms
Speed: 5.7ms preprocess, 354.7ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



...

[374, 242]


0: 320x640 1 person, 366.2ms
Speed: 3.4ms preprocess, 366.2ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 person, 471.7ms
Speed: 2.8ms preprocess, 471.7ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.



[376, 243]


0: 320x640 1 person, 377.7ms
Speed: 3.5ms preprocess, 377.7ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 person, 411.7ms
Speed: 3.3ms preprocess, 411.7ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 person, 379.9ms
Speed: 4.7ms preprocess, 379.9ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 person, 375.1ms
Speed: 3.9ms preprocess, 375.1ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 320x640 1 person, 341.5ms
Speed: 4.2ms preprocess, 341.5ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


[377, 243]
[378, 243]
[379, 243]
[380, 243]
[385, 238]
[387, 218]
[390, 217]
[396, 207]
[410, 199]
[421, 190]
[428, 185]
[431, 182]
[432, 182]
[433, 181]
[434, 181]
[436, 180]
[438, 177]
[439, 177]
[439, 176]
[444, 172]
[456, 169]
[466, 166]
[481, 161]
[502, 159]
[524, 149]
[552, 149]
[580, 153]
[616, 155]
[648, 159]
[672, 161]
[698, 167]
[726, 167]
[737, 172]
[748, 177]
[754, 178]
[754, 180]
[756, 181]
[747, 187]
[727, 204]
[709, 220]
[689, 236]
[665, 256]
[639, 268]
[626, 278]
[615, 286]
[610, 288]
[609, 288]
[609, 289]
[611, 293]
[613, 297]
[635, 305]
[643, 319]
[643, 323]
[643, 324]
[644, 324]
[645, 324]
[646, 324]
[646, 323]
[646, 321]
[649, 318]
[653, 308]
[656, 301]
[656, 300]
[656, 301]
[660, 303]
[690, 310]
[712, 322]
[744, 334]
[766, 344]
[779, 348]
[786, 348]
[790, 349]
[796, 348]
[813, 343]
[828, 335]
[844, 328]
[857, 319]
[871, 314]
[877, 309]
[889, 303]
[897, 297]
[898, 296]
[899, 293]
[900, 290]
[902, 284]
[900, 266]
[916, 252]
[920, 274]
[925, 279]
[922, 277]
[920, 277]